In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

from transformers import BertTokenizer
import keras
from tqdm.notebook import tqdm
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
df = pd.read_csv('C:/Users/User/Desktop/df_merged.csv', header=0)
df.head()

,Unnamed: 0,Vertex 1,Vertex 2,Relationship,Relationship Date (UTC),Tweet,URLs in Tweet,Domains in Tweet,Hashtags in Tweet,Media in Tweet,...,Default Profile,Default Profile Image,Geo Enabled,Language_y,Listed Count,Profile Background Image Url,Verified,Custom Menu Item Text,Custom Menu Item Action,Tweeted Search Term?
0,0,bluebottleroast,bluebottleroast,Tweet,2021-09-21 16:01:49,We’re thrilled to announce Blue Bottle commits...,https://blog.bluebottlecoffee.com/posts/carbon...,bluebottlecoffee.com,NaN,NaN,...,False,False,True,NaN,1641,https://abs.twimg.com/images/themes/theme1/bg.png,True,Open Twitter Page for This Person,https://twitter.com/bluebottleroast,Yes
1,1,bacademicsorg,bacademicsorg,Tweet,2021-12-09 17:00:53,Call for paper. Join the first edition of the ...,NaN,NaN,NaN,https://pbs.twimg.com/media/FGLlu_AXwA0zsXK.jpg,...,True,False,False,NaN,7,NaN,False,Open Twitter Page for This Person,https://twitter.com/bacademicsorg,Yes
2,2,bluwndzdancing,bluwndzdancing,Tweet,2021-12-11 23:27:47,Please consider signing this letter to EGLE to...,https://www.miclimateaction.org/sign_on_for_a_...,miclimateaction.org,NaN,NaN,...,False,False,True,NaN,11,https://abs.twimg.com/images/themes/theme8/bg.gif,False,Open Twitter Page for This Person,https://twitter.com/bluwndzdancing,Yes
3,3,zyitegadgets,zyitegadgets,Tweet,2021-12-11 23:58:08,For Hong Kong to achieve carbon neutrality by ...,https://www.scmp.com/comment/opinion/article/3...,scmp.com,NaN,https://pbs.twimg.com/media/FGXYbRiVUAIOmQj.jpg,...,True,False,False,NaN,155,https://abs.twimg.com/images/themes/theme1/bg.png,False,Open Twitter Page for This Person,https://twitter.com/zyitegadgets,Yes
4,4,jlitwinetz,jlitwinetz,Tweet,2021-12-12 00:18:05,For Hong Kong to achieve carbon neutrality by ...,https://www.scmp.com/comment/opinion/article/3...,scmp.com,NaN,NaN,...,False,False,False,NaN,17,https://abs.twimg.com/images/themes/theme1/bg.png,False,Open Twitter Page for This Person,https://twitter.com/jlitwinetz,Yes


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 87 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      1008 non-null   int64  
 1   Vertex 1                        1008 non-null   object 
 2   Vertex 2                        1008 non-null   object 
 3   Relationship                    1008 non-null   object 
 4   Relationship Date (UTC)         1008 non-null   object 
 5   Tweet                           1008 non-null   object 
 6   URLs in Tweet                   877 non-null    object 
 7   Domains in Tweet                877 non-null    object 
 8   Hashtags in Tweet               228 non-null    object 
 9   Media in Tweet                  589 non-null    object 
 10  Tweet Image File                1008 non-null   object 
 11  Tweet Date (UTC)                1008 non-null   object 
 12  Date                            10

In [37]:
df = data[data['Vertex 1'] == data['Vertex 2']]
df = df[df['Language']== 'en']
df = df.reset_index()
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,index,Unnamed: 0,Vertex 1,Vertex 2,Relationship,Relationship Date (UTC),Tweet,URLs in Tweet,Domains in Tweet,Hashtags in Tweet,...,Default Profile,Default Profile Image,Geo Enabled,Language_y,Listed Count,Profile Background Image Url,Verified,Custom Menu Item Text,Custom Menu Item Action,Tweeted Search Term?


In [42]:
df['Tweet']

0       We’re thrilled to announce Blue Bottle commits...
1       Call for paper. Join the first edition of the ...
2       Please consider signing this letter to EGLE to...
3       For Hong Kong to achieve carbon neutrality by ...
4       For Hong Kong to achieve carbon neutrality by ...
                              ...                        
1003    ATTENTION LAST MINUTE SHOPPERS: Give the gift ...
1004    #Columbus is facing Climate hazards like risin...
1005    By the end of 2019, only 67 countries had anno...
1006    Tech giant Alibaba commits to becoming carbon ...
1007    #ICYMI: Toyota has some electrifying news abou...
Name: Tweet, Length: 1008, dtype: object

In [43]:
test = pd.DataFrame(df['Tweet'])
test.columns = ['text']

In [44]:
text = []
for txt in test['text']:
    txt = re.sub(r'http\S+', '', txt)
    text.append(str(txt))
test = pd.DataFrame(text, columns=['text'])
test

,text
0,We’re thrilled to announce Blue Bottle commits...
1,Call for paper. Join the first edition of the ...
2,Please consider signing this letter to EGLE to...
3,For Hong Kong to achieve carbon neutrality by ...
4,For Hong Kong to achieve carbon neutrality by ...
...,...
1003,ATTENTION LAST MINUTE SHOPPERS: Give the gift ...
1004,#Columbus is facing Climate hazards like risin...
1005,"By the end of 2019, only 67 countries had anno..."
1006,Tech giant Alibaba commits to becoming carbon ...


In [45]:
#pre-trained model
model = keras.models.load_model("colbert-trained/")
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, 20)]         0                                            
_______________________________________________________________________________________

In [46]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        #sent
        sentences = sent_tokenize(row.text)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        #full row
        ids_q, masks_q, segments_q = return_id(row.text, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input

In [47]:
#!pip install transformers --user

In [48]:
MODEL_TYPE = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

In [49]:
input_categories = list(test.columns[[0]])
input_categories

['text']

In [50]:
MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [51]:
test_inputs = compute_input_arrays(test, input_categories, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



(1008, 20)


In [52]:
test_inputs

[array([[  101,  2057,  1521, ...,  1998,  2057,   102],
        [  101,  2655,  2005, ...,     0,     0,     0],
        [  101,  3531,  5136, ...,  6351, 21083,   102],
        ...,
        [  101,  2011,  1996, ...,  6351, 21083,   102],
        [  101,  6627,  5016, ...,     0,     0,     0],
        [  101,  1001, 13580, ...,  6046,  3751,   102]]),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[  101,  2023,  2627, ..., 18622, 11916,   102],
        [  101,  3693,  1996, ..., 11703, 11387,   102],
        [  101,  2023,  2003, ...,   999,   102,     0],
        ...,
        [  101,  2085,  1010, ...

In [53]:
test_preds = model.predict(test_inputs)

In [54]:
test_preds

array([[0.00134906],
       [0.9997157 ],
       [0.15550995],
       ...,
       [0.6520544 ],
       [0.0762307 ],
       [0.30806166]], dtype=float32)

In [55]:
df2 = pd.DataFrame(test_preds, columns=['humor'])
df2

,humor
0,0.001349
1,0.999716
2,0.155510
3,0.000109
4,0.000109
...,...
1003,0.991250
1004,0.948394
1005,0.652054
1006,0.076231


In [19]:
#df2.to_csv('out.csv')

In [56]:
#merge dataframe
df_merge = pd.concat([df, df2], axis=1)
df_merge

,Unnamed: 0,Vertex 1,Vertex 2,Relationship,Relationship Date (UTC),Tweet,URLs in Tweet,Domains in Tweet,Hashtags in Tweet,Media in Tweet,...,Default Profile Image,Geo Enabled,Language_y,Listed Count,Profile Background Image Url,Verified,Custom Menu Item Text,Custom Menu Item Action,Tweeted Search Term?,humor
0,0,bluebottleroast,bluebottleroast,Tweet,2021-09-21 16:01:49,We’re thrilled to announce Blue Bottle commits...,https://blog.bluebottlecoffee.com/posts/carbon...,bluebottlecoffee.com,NaN,NaN,...,False,True,NaN,1641,https://abs.twimg.com/images/themes/theme1/bg.png,True,Open Twitter Page for This Person,https://twitter.com/bluebottleroast,Yes,0.001349
1,1,bacademicsorg,bacademicsorg,Tweet,2021-12-09 17:00:53,Call for paper. Join the first edition of the ...,NaN,NaN,NaN,https://pbs.twimg.com/media/FGLlu_AXwA0zsXK.jpg,...,False,False,NaN,7,NaN,False,Open Twitter Page for This Person,https://twitter.com/bacademicsorg,Yes,0.999716
2,2,bluwndzdancing,bluwndzdancing,Tweet,2021-12-11 23:27:47,Please consider signing this letter to EGLE to...,https://www.miclimateaction.org/sign_on_for_a_...,miclimateaction.org,NaN,NaN,...,False,True,NaN,11,https://abs.twimg.com/images/themes/theme8/bg.gif,False,Open Twitter Page for This Person,https://twitter.com/bluwndzdancing,Yes,0.155510
3,3,zyitegadgets,zyitegadgets,Tweet,2021-12-11 23:58:08,For Hong Kong to achieve carbon neutrality by ...,https://www.scmp.com/comment/opinion/article/3...,scmp.com,NaN,https://pbs.twimg.com/media/FGXYbRiVUAIOmQj.jpg,...,False,False,NaN,155,https://abs.twimg.com/images/themes/theme1/bg.png,False,Open Twitter Page for This Person,https://twitter.com/zyitegadgets,Yes,0.000109
4,4,jlitwinetz,jlitwinetz,Tweet,2021-12-12 00:18:05,For Hong Kong to achieve carbon neutrality by ...,https://www.scmp.com/comment/opinion/article/3...,scmp.com,NaN,NaN,...,False,False,NaN,17,https://abs.twimg.com/images/themes/theme1/bg.png,False,Open Twitter Page for This Person,https://twitter.com/jlitwinetz,Yes,0.000109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,1003,carbonfundorg,carbonfundorg,Tweet,2021-12-20 17:11:48,ATTENTION LAST MINUTE SHOPPERS: Give the gift ...,https://carbonfund.org/donate/,carbonfund.org,NaN,https://pbs.twimg.com/media/FHERTRIWYAwqzhZ.png,...,False,False,NaN,283,https://abs.twimg.com/images/themes/theme1/bg.png,False,Open Twitter Page for This Person,https://twitter.com/carbonfundorg,Yes,0.991250
1004,1004,citiesforum_org,citiesforum_org,Tweet,2021-12-20 16:03:35,#Columbus is facing Climate hazards like risin...,https://www.smartcitiesworld.net/news/columbus...,smartcitiesworld.net,columbus urbanplanning sustainability cityplan...,NaN,...,False,False,NaN,110,NaN,False,Open Twitter Page for This Person,https://twitter.com/citiesforum_org,Yes,0.948394
1005,1005,undpclimate,undpclimate,Tweet,2021-12-14 13:32:38,"By the end of 2019, only 67 countries had anno...",https://climatepromise.undp.org/news-and-stori...,undp.org,netzero,NaN,...,False,True,NaN,739,https://abs.twimg.com/images/themes/theme1/bg.png,True,Open Twitter Page for This Person,https://twitter.com/undpclimate,Yes,0.652054
1006,1006,greenpeaceeasia,greenpeaceeasia,Tweet,2021-12-20 09:19:22,Tech giant Alibaba commits to becoming carbon ...,https://www.greenpeace.org/eastasia/press/7123...,greenpeace.org,climateactionnow,NaN,...,False,True,NaN,558,https://abs.twimg.com/images/themes/theme13/bg...,True,Open Twitter Page for This Person,https://twitter.com/greenpeaceeasia,Yes,0.076231


In [57]:
df_merge['Retweet_Count'] = df_merge['Retweet Count']
df_merge['Favorite_Count'] = df_merge['Favorite Count']

In [58]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [59]:
model1 = smf.ols(formula='Retweet_Count ~ humor ', data = df_merge).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Retweet_Count   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     17.34
Date:                Sat, 08 Jan 2022   Prob (F-statistic):           3.39e-05
Time:                        23:00:27   Log-Likelihood:                -4641.8
No. Observations:                1008   AIC:                             9288.
Df Residuals:                    1006   BIC:                             9298.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1408      0.902      1.265      0.206      -0.628       2.910
humor          8.1635      1.960      4.164      0.000       4.317      12.010
==============================================================================
Omnibus:                     2029.052   Durbin-Watson:                   1.904
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3155027.049
Skew:                          15.529   Prob(JB):                         0.00
Kurtosis:                     275.314   Cond. No.                         2.75
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
df_merge.to_csv('df_merged_for_r.csv')